In [1]:
import pandas as pd
from darts import TimeSeries
import matplotlib.pyplot as plt
import numpy as np

In [49]:
data1 = pd.read_excel("raw_Cp33a45b6b_a.xlsx")
data2 = pd.read_excel("raw_Cp5_a.xlsx")
data3 = pd.read_excel("raw_Cp10_a.xlsx")


In [50]:
def give_carpark_name(x):
    name_dict = {48:"CP6B",52:"CP6B",70:"CP5",76:"CP4",82:"CP3",83:"CP3",92:"5B",161:"3A"}
    return name_dict[x]

In [51]:
joint1 = pd.concat([data1,data2],ignore_index=True)


In [52]:
joint1['CP'] = joint1["ExitId"].apply(give_carpark_name)

In [53]:
data3 = data3.rename(columns={"Enter Time":"enter","Exit time":"Exit"}).drop(columns = ['report time'])
data3["CP"] = "CP10"

In [54]:
joint1 = joint1.drop(columns = ['ExitId'])

In [58]:
data = pd.concat([joint1,data3],ignore_index=True)

In [59]:
#correcting inconsistencies in time entries
def correct_time(data,timecol,timebool):
    temp = []
    for i in range(len(data)):
        if not data[timebool].iloc[i]:
            x = pd.to_datetime(data[timecol].iloc[i], format='%d/%m/%Y %H:%M').round(freq='T')
        else:
            x = pd.to_datetime(data[timecol].iloc[i], format='%Y-%d-%m %H:%M:%S').round(freq='T')

        temp.append(x)

    return temp    
    

In [60]:
data["enter_dt"] = correct_time(data,"enter","enter_formatted")
data["exit_dt"] = correct_time(data,"Exit","exit_formatted")

In [62]:
data["enter_dt"] = data["enter_dt"].apply(lambda x: x.replace(year = 2023) if x.year == 2037 else x)
data["enter_dt"] = data["enter_dt"].apply(lambda x: x.replace(year = 2022) if x.year == 2026 else x)

In [63]:
data = data[(data["enter_dt"].dt.year >= 2022) & (data["exit_dt"].dt.year >= 2022)]

In [64]:
print(data["enter_dt"].min())
print(data["enter_dt"].max())
print(data["exit_dt"].min())
print(data["exit_dt"].max())

2022-04-01 00:00:00
2023-04-30 23:56:00
2022-04-01 00:00:00
2023-04-30 23:59:00


In [65]:
data.head()

,IU,enter,enter_formatted,Exit,exit_formatted,hourly_du,staff_du,student_du,esp_du,CP,enter_dt,exit_dt
0,739d68a9,2022-01-07 18:17:00,True,2022-01-07 22:21:00,True,244,\N,\N,\N,CP6B,2022-07-01 18:17:00,2022-07-01 22:21:00
1,0,2022-02-07 00:00:00,True,2022-02-07 00:00:00,True,0,\N,\N,\N,CP6B,2022-07-02 00:00:00,2022-07-02 00:00:00
2,28bde674,2022-01-07 22:22:00,True,2022-02-07 00:13:00,True,111,\N,\N,\N,CP6B,2022-07-01 22:22:00,2022-07-02 00:13:00
3,336c1bbe,2022-01-07 22:53:00,True,2022-02-07 01:07:00,True,134,\N,\N,\N,CP6B,2022-07-01 22:53:00,2022-07-02 01:07:00
4,26dfa98b,2022-02-07 08:01:00,True,2022-02-07 08:02:00,True,1,\N,\N,\N,CP6B,2022-07-02 08:01:00,2022-07-02 08:02:00


In [66]:
data = data[["CP","IU","enter_dt","exit_dt"]].drop_duplicates()

In [68]:
data = data[data["exit_dt"] > data["enter_dt"]]

In [78]:
enter_df_all = data[["CP","IU","enter_dt"]]
exit_df_all = data[["CP","IU","exit_dt"]]

In [79]:
enter_df_all.sort_values(by = ["CP","enter_dt"],inplace=True)
exit_df_all.sort_values(by = ["CP","exit_dt"],inplace=True)


C:\Users\dxcas\AppData\Local\Temp\ipykernel_21408\2137482607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enter_df_all.sort_values(by = ["CP","enter_dt"],inplace=True)
C:\Users\dxcas\AppData\Local\Temp\ipykernel_21408\2137482607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_df_all.sort_values(by = ["CP","exit_dt"],inplace=True)


In [81]:
change_data_frame = pd.DataFrame(columns = ["dt","CP","delta","delta_nett"])

for i in data["CP"].drop_duplicates():
    enter_df = enter_df_all[enter_df_all["CP"] == i]
    exit_df = exit_df_all[enter_df_all["CP"] == i]
    # Log number of entries/exits at each timestamp
    enter_df = enter_df.groupby("enter_dt").size().reset_index(name = "delta")
    exit_df = exit_df.groupby("exit_dt").size().reset_index(name = "delta")
    enter_df["dt"] = enter_df["enter_dt"]
    exit_df["dt"] = exit_df["exit_dt"]
    enter_df["delta_nett"] = enter_df["delta"]
    exit_df["delta_nett"] = exit_df["delta"] * -1

    start_datetime_enter = enter_df["dt"].min()
    end_datetime_enter = enter_df["dt"].max()
    minute_range_enter = pd.date_range(start = start_datetime_enter, end = end_datetime_enter, freq = "T")
    temp_df = pd.DataFrame({'dt':minute_range_enter})
    enter_df = pd.merge(temp_df,enter_df, on = 'dt', how = 'left').fillna(0).drop('enter_dt',axis = 1)

    start_datetime_exit = exit_df["dt"].min()
    end_datetime_exit = exit_df["dt"].max()
    minute_range_exit = pd.date_range(start = start_datetime_exit, end = end_datetime_exit, freq = "T")
    temp_df = pd.DataFrame({'dt':minute_range_exit})
    exit_df = pd.merge(temp_df,exit_df, on = 'dt', how = 'left').fillna(0).drop('exit_dt',axis = 1)

    enter_df["CP"] = i
    exit_df["CP"] = i

    change_data_frame = pd.concat([change_data_frame,enter_df,exit_df],ignore_index=True)

C:\Users\dxcas\AppData\Local\Temp\ipykernel_21408\525524465.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exit_df = exit_df_all[enter_df_all["CP"] == i]
C:\Users\dxcas\AppData\Local\Temp\ipykernel_21408\525524465.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exit_df = exit_df_all[enter_df_all["CP"] == i]
C:\Users\dxcas\AppData\Local\Temp\ipykernel_21408\525524465.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exit_df = exit_df_all[enter_df_all["CP"] == i]
C:\Users\dxcas\AppData\Local\Temp\ipykernel_21408\525524465.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exit_df = exit_df_all[enter_df_all["CP"] == i]
C:\Users\dxcas\AppData\Local\Temp\ipykernel_21408\525524465.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exit_df = exit_df_all[enter_df_all["CP"] == i]
C:\Users\dxcas\AppData\Local\Temp\ipykernel_2

In [83]:
nett_df = change_data_frame[["CP","dt","delta_nett"]]

In [89]:
nett_df_adj = nett_df.groupby(["CP","dt"]).sum("delta_nett").reset_index().sort_values(["CP","dt"])
#nett_df_adj["capacity"] = nett_df_adj["delta_nett"].cumsum()

In [93]:
nett_df_adj['num_occupied'] = nett_df_adj.groupby("CP")["delta_nett"].cumsum()

In [94]:
nett_df_adj

,CP,dt,delta_nett,num_occupied
0,3A,2022-04-01 04:02:00,1.0,1.0
1,3A,2022-04-01 04:03:00,0.0,1.0
2,3A,2022-04-01 04:04:00,0.0,1.0
3,3A,2022-04-01 04:05:00,0.0,1.0
4,3A,2022-04-01 04:06:00,0.0,1.0
...,...,...,...,...
3218600,CP6B,2023-04-30 22:43:00,0.0,1.0
3218601,CP6B,2023-04-30 22:44:00,0.0,1.0
3218602,CP6B,2023-04-30 22:45:00,0.0,1.0
3218603,CP6B,2023-04-30 22:46:00,0.0,1.0


In [103]:
sample = nett_df_adj[nett_df_adj["CP"] == "CP4"]
series = TimeSeries.from_dataframe(sample,time_col="dt",value_cols="num_occupied")

In [106]:
nett_df_adj.to_csv("sample_data_frontend.csv")